# `ProteinWorkshop` Tutorial, Part 1 - Training a New Model
![Models](../docs/source/_static/box_models.png)

## Train a new model using the `ProteinWorkshop`

In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

Welcome to the tutorial series for the `ProteinWorkshop`! 

In the `ProteinWorkshop`, we implement numerous [featurisation](https://www.proteins.sh/configs/features) schemes, [datasets](https://www.proteins.sh/configs/dataset) for [self-supervised pre-training](https://proteins.sh/quickstart_component/pretrain.html) and [downstream evaluation](https://proteins.sh/quickstart_component/downstream.html), [pre-training](https://proteins.sh/configs/task) tasks, and [auxiliary tasks](https://proteins.sh/configs/task.html#auxiliary-tasks).

[Processed datasets](https://drive.google.com/drive/folders/18i8rLST6ZICTBu6Q67ClT0KqN9AHeqoW?usp=sharing) and [pre-trained weights](https://drive.google.com/drive/folders/1zK1r8FpmGaqV_QwUJuvDacwSL0RW-Vw9?usp=sharing) are made available. Downloading datasets is not required; upon first run all datasets will be downloaded and processed from their respective source.

The `ProteinWorkshop` encompasses several models as well as pre-trained weights for them so that you can readily use them.

In this tutorial, we show you how you can use what is already available in the protein workshop to train and use models for specific tasks. The `ProteinWorkshop` is structured as a very modular package; we will therefore talk about how to change the different parts of it, like the model, training task, dataset, featurization scheme, etc. in this tutorial. 

Besides using all the different options we provide, you can make use of the modular nature of the `ProteinWorkshop` to add your own models, datasets, featurization schemes, and training tasks. We will show you how to do this in the next tutorials.

To train a new model, you can follow the following 3-step procedure:

1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks
2. Validate the designed training config
3. Use the designed config to train a new model

### 1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks

You can switch out any of these for another available option by replacing the corresponding argument's value in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=afdb_swissprot_v4", "features=ca_base", "+aux_task=none"], return_hydra_config=True)`

In [2]:
import os

# Set the environment variable to make only GPU 0 visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Now import the necessary librarie
import torch
# Verify GPUs visible to PyTorch
torch_visible_gpus = torch.cuda.device_count()
print("Visible GPUs for PyTorch:", torch_visible_gpus)

Visible GPUs for PyTorch: 1


In [3]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")
# print(rel_path)
GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=schnet",
        "encoder.hidden_channels=512", # Number of channels in the hidden layers
        "encoder.out_dim=32", # Output dimension of the model
        "encoder.num_layers=6", # Number of filters used in convolutional layers
        "encoder.num_filters=128", # Number of convolutional layers in the model
        "encoder.num_gaussians=50", # Number of Gaussian functions used for radial filters
        "encoder.cutoff=10.0", # Cutoff distance for interactions
        "encoder.max_num_neighbors=32", # Maximum number of neighboring atoms to consider
        "encoder.readout=add", # Global pooling method to be used
        "encoder.dipole=False",
        "encoder.mean=null",
        "encoder.std=null",
        "encoder.atomref=null",
        "encoder.pretraining=False",

        "task=multiclass_graph_classification",
        "dataset=fold_family",
        "dataset.datamodule.batch_size=32",
        "features=ca_angles",  # here
        "+aux_task=none",
        "callbacks.model_checkpoint.save_top_k=-1",
        "trainer.max_epochs=150",
        "optimiser=adam",
        "optimiser.optimizer.lr=3e-4",
        "callbacks.early_stopping.patience=10",
        "test=True",
        "scheduler=plateau", # 5 epochs - 0.6 default
        # "+ckpt_path=/home/zhang/Projects/3d/proteinworkshop_checkpoints/outputs_schnet_pretraining_best@0/checkpoints/last.ckpt", # continue training
        ## for test ONLY
        # "task_name=test",  # here
        # "ckpt_path_test=/home/zhang/Projects/3d/ProteinWorkshop/notebooks/outputs/checkpoints/epoch_016.ckpt", # here
        # "optimizer.weight_decay=0.5"
        "seed=66",
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[07/30/24 02:23:19] DEBUG    No `.env` file found in project root. Checking for env vars...         ]8;id=375770;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=2729;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#22\22]8;;\

                    DEBUG    No env var `DATA_PATH` found. Setting default...                       ]8;id=390722;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=17984;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#28\28]8;;\

                    INFO     DATA_PATH:                                                             ]8;id=329389;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py\constants.py]8;;\:]8;id=29926;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/constants.py#39\39]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/data                          

[07/30/24 02:23:20] INFO     Hydra initialised at                                                    ]8;id=897046;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=59396;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/config.                       

OverrideParseException: mismatched input '=' expecting <EOF>
See https://hydra.cc/docs/1.2/advanced/override_grammar/basic for details

### 2. Validate the designed training config

This is not strictly necessary, but it is a good idea to validate the config before training. This will check that all the arguments you have provided are valid and that the config is complete.

In [ ]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

                    DEBUG    CUDA available: True                                                     ]8;id=505185;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=786614;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=173370;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=194629;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=982041;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=880076;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=618808;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=585041;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [ ]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'hparams', 'name', 'seed', 'num_workers', 'task_name', 'ckpt_path_test', 'test', 'aux_task'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from tho

### 3. Use the designed config to train a new model

Now with the config you have designed, you can train a new model. You can also use the `ProteinWorkshop` to evaluate the model on a downstream task.

In [ ]:
from proteinworkshop.train import train_model

train_model(cfg)

Seed set to 43


[07/28/24 16:31:14] INFO     Instantiating datamodule:                                                  ]8;id=387844;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=704272;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#99\99]8;;\
                             <proteinworkshop.datasets.fold_classification.FoldClassificationDataModule            
                             ...                                                                                   

                    INFO     Instantiating callbacks...                                                ]8;id=522476;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=638570;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#106\106]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>   ]8;id=893689;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=993108;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.EarlyStopping>     ]8;id=455153;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=413071;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

...

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1608. To avoid any 
miscalculations, use `self.log(..., batch_size=batch_size)`.

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2552. To avoid any 
miscalculations, use `self.log(..., batch_size=batch_size)`.

/home/zhang/miniconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: UserWarning:

Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 300. To avoid any 
miscalculations, use `self.log(..., batch_size=batch_size)`.

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ test/graph_label/accuracy │    0.33140623569488525    │
│  test/graph_label/f1_max  │    0.4003984034061432     │
│ test/graph_label/f1_score │    0.11053196340799332    │
│   test/loss/graph_label   │    12.169452667236328     │
│      test/loss/total      │    12.169452667236328     │
└───────────────────────────┴───────────────────────────┘

[07/28/24 17:56:02] INFO     superfamily: {'test/loss/graph_label/superfamily': 12.169452667236328,    ]8;id=85046;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=759789;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#204\204]8;;\
                             'test/loss/total/superfamily': 12.169452667236328,                                    
                             'test/graph_label/accuracy/superfamily': 0.33140623569488525,                         
                             'test/graph_label/f1_score/superfamily': 0.11053196340799332,                         
                             'test/graph_label/f1_max/superfamily': 0.4003984034061432}                            

### 4. Wrapping up

Have any additional questions about using the components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py